In [ ]:
install.packages("ggplot2")


: 

In [ ]:
load("..\..\data\wage2015_subsample_inference.csv")

# Print the structure of the data frame
str(data)

# Print a summary of the data frame
print("Summary of the data:")
summary(data)

# Print dimensions of the data frame
print("Dimensions of the data:")
print(dim(data))


: 

In [9]:
flex <- lwage ~ sex + shs + hsg + scl + clg + occ2 + ind2 + mw + so + we +
  (exp1 + exp2 + exp3 + exp4) * (shs + hsg + scl + clg + occ2 + ind2 + mw + so + we)

# Fit the model using lm() function for OLS regression
flex_results <- lm(flex, data=data)

summary(flex_results)



Call:
lm(formula = flex, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.1282 -0.3065 -0.0151  0.2945  3.5341 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.5459818  0.1293099  27.422  < 2e-16 ***
sex         -0.1024693  0.0146380  -7.000 2.89e-12 ***
shs         -0.6909668  0.8988057  -0.769  0.44207    
hsg         -0.5816131  0.1944598  -2.991  0.00279 ** 
scl         -0.3297645  0.1234560  -2.671  0.00758 ** 
clg         -0.0873795  0.0668027  -1.308  0.19092    
occ2        -0.0266617  0.0050951  -5.233 1.74e-07 ***
ind2        -0.0161333  0.0062181  -2.595  0.00950 ** 
mw           0.1079623  0.0834048   1.294  0.19557    
so           0.0385942  0.0747084   0.517  0.60546    
we          -0.0035042  0.0854161  -0.041  0.96728    
exp1         0.0395328  0.0481220   0.822  0.41139    
exp2        -0.0470984  0.5325375  -0.088  0.92953    
exp3        -0.0771429  0.2154298  -0.358  0.72029    
exp4         0.0192883  0.0

In [10]:

# Get exogenous variables from the flexible model
X <- model.matrix(flex_results)

# Set endogenous variable
y <- data$lwage  # Directly from the data frame

# Alternatively, extracting response variable from the model object
y_model <- model.response(model.frame(flex_results))

# Verify the contents
head(X)  # Shows the first few rows of the model matrix
head(y)  # Shows the first few values of the response variable


(Intercept),sex,shs,hsg,scl,clg,occ2,ind2,mw,so,...,we:exp3,shs:exp4,hsg:exp4,scl:exp4,clg:exp4,occ2:exp4,ind2:exp4,mw:exp4,so:exp4,we:exp4
1,1,0,0,0,1,11,18,0,0,...,0,0,0.0000,0,0.2401,2.6411,4.3218,0,0,0
1,0,0,0,0,1,10,9,0,0,...,0,0,0.0000,0,92.3521,923.5210,831.1689,0,0,0
1,0,0,1,0,0,19,4,0,0,...,0,0,10.4976,0,0.0000,199.4544,41.9904,0,0,0
1,1,0,0,0,0,1,12,0,0,...,0,0,0.0000,0,0.0000,39.0625,468.7500,0,0,0
1,1,0,0,0,1,6,22,0,0,...,0,0,0.0000,0,23.4256,140.5536,515.3632,0,0,0
1,1,0,0,0,1,5,14,0,0,...,0,0,0.0000,0,0.0001,0.0005,0.0014,0,0,0


[1] 2.263364 3.872802 2.403126 2.634928 3.361977 2.462215

In [15]:
set.seed(24)  # For reproducibility

# Calculate the number of observations to split on
n <- nrow(X)
train_size <- floor(0.8 * n)

# Randomly sample indices for the training data
train_indices <- sample(seq_len(n), size = train_size)

# Create training and test datasets
X_train <- X[train_indices, , drop = FALSE]
X_test <- X[-train_indices, , drop = FALSE]
y_train <- y[train_indices]
y_test <- y[-train_indices]


In [16]:
# Calculate the mean and standard deviation from the training set
train_mean <- apply(X_train, 2, mean)
train_sd <- apply(X_train, 2, sd)

# Standardize the training data
X_train_scaled <- sweep(X_train, 2, train_mean, FUN = "-")
X_train_scaled <- sweep(X_train_scaled, 2, train_sd, FUN = "/")

# Standardize the test data using the same mean and standard deviation as calculated from the training set
X_test_scaled <- sweep(X_test, 2, train_mean, FUN = "-")
X_test_scaled <- sweep(X_test_scaled, 2, train_sd, FUN = "/")

# Check the results of scaling
head(X_train_scaled)
head(X_test_scaled)


,(Intercept),sex,shs,hsg,scl,clg,occ2,ind2,mw,so,...,we:exp3,shs:exp4,hsg:exp4,scl:exp4,clg:exp4,occ2:exp4,ind2:exp4,mw:exp4,so:exp4,we:exp4
2791,NaN,-0.9017665,-0.1485716,-0.5731701,-0.6169316,1.4586999,-1.5310333,-0.07070352,-0.5873694,1.5362126,...,-0.2327138,-0.05283426,-0.2453471,-0.2401136,-0.2146948,-0.3891810,-0.4238188,-0.22379577,-0.2210076,-0.1989129
1239,NaN,1.1086653,-0.1485716,-0.5731701,-0.6169316,1.4586999,-1.3875261,-0.24650224,1.7020930,-0.6507936,...,-0.2327138,-0.05283426,-0.2453471,-0.2401136,0.1249279,-0.3695401,-0.2932612,0.08194768,-0.2212557,-0.1989129
2466,NaN,-0.9017665,-0.1485716,-0.5731701,1.6205317,-0.6853756,1.3391099,1.33568627,1.7020930,-0.6507936,...,-0.2327138,-0.05283426,-0.2453471,-0.1445640,-0.2150245,-0.3186339,-0.3457439,-0.11924316,-0.2212557,-0.1989129
2979,NaN,-0.9017665,-0.1485716,-0.5731701,-0.6169316,1.4586999,0.6215741,-0.77389841,-0.5873694,1.5362126,...,-0.2327138,-0.05283426,-0.2453471,-0.2401136,-0.2052518,-0.3846713,-0.4211382,-0.22379577,-0.2139012,-0.1989129
1949,NaN,1.1086653,-0.1485716,-0.5731701,-0.6169316,1.4586999,0.7650812,1.51148499,1.7020930,-0.6507936,...,-0.2327138,-0.05283426,-0.2453471,-0.2401136,1.3749219,0.3919939,0.6966795,1.20615642,-0.2212557,-0.1989129
906,NaN,-0.9017665,-0.1485716,-0.5731701,-0.6169316,-0.6853756,-1.2440190,-0.24650224,-0.5873694,-0.6507936,...,-0.2327138,-0.05283426,-0.2453471,-0.2401136,-0.2150245,-0.3521432,-0.2596881,-0.22379577,-0.2212557,-0.1989129


,(Intercept),sex,shs,hsg,scl,clg,occ2,ind2,mw,so,...,we:exp3,shs:exp4,hsg:exp4,scl:exp4,clg:exp4,occ2:exp4,ind2:exp4,mw:exp4,so:exp4,we:exp4
3,NaN,-0.9017665,-0.1485716,1.7442594,-0.6169316,-0.6853756,1.0520955,-1.6528920,-0.5873694,-0.6507936,...,-0.2327138,-0.05283426,0.0324982,-0.2401136,-0.2150245,-0.1545580,-0.3692001,-0.2237958,-0.2212557,-0.1989129
11,NaN,1.1086653,-0.1485716,1.7442594,-0.6169316,-0.6853756,0.1910526,0.8082901,-0.5873694,-0.6507936,...,-0.2327138,-0.05283426,-0.2389923,-0.2401136,-0.2150245,-0.3855187,-0.4183204,-0.2237958,-0.2212557,-0.1989129
12,NaN,-0.9017665,-0.1485716,1.7442594,-0.6169316,-0.6853756,0.3345598,0.8082901,-0.5873694,-0.6507936,...,-0.2327138,-0.05283426,1.8985211,-0.2401136,-0.2150245,0.9448152,1.4772938,-0.2237958,-0.2212557,-0.1989129
17,NaN,-0.9017665,-0.1485716,1.7442594,-0.6169316,-0.6853756,1.0520955,-1.6528920,-0.5873694,-0.6507936,...,-0.2327138,-0.05283426,1.1612448,-0.2401136,-0.2150245,0.7986367,-0.1467538,-0.2237958,-0.2212557,-0.1989129
22,NaN,1.1086653,-0.1485716,-0.5731701,1.6205317,-0.6853756,-0.0959617,0.8082901,-0.5873694,-0.6507936,...,-0.2327138,-0.05283426,-0.2453471,-0.2226501,-0.2150245,-0.3824357,-0.4117034,-0.2237958,-0.2212557,-0.1989129
29,NaN,-0.9017665,-0.1485716,-0.5731701,-0.6169316,1.4586999,0.7650812,-0.7738984,-0.5873694,-0.6507936,...,-0.2327138,-0.05283426,-0.2453471,-0.2401136,-0.1983527,-0.3809992,-0.4191490,-0.2237958,-0.2212557,-0.1989129


In [17]:
alphas <- seq(0.1, 0.5, by = 0.1)

In [19]:
# Set seed for reproducibility
set.seed(24)

# Assuming 'data' is your dataset and you want to create indices for 5 folds
n <- nrow(data)  # Get the number of rows in the dataset
indices <- sample(seq_len(n))  # Shuffle the data indices

# Create 5 equally sized folds
folds <- cut(indices, breaks=5, labels=FALSE)  # Assign each index to one of 5 folds


In [20]:
train_indices <- which(folds != 1)  # All but the first fold for training
test_indices <- which(folds == 1)   # The first fold for testing

# Extract training and testing data based on indices
train_data <- data[train_indices, ]
test_data <- data[test_indices, ]


In [21]:
lasso_regression <- function(X_train, y_train, alpha, iterations = 100, learning_rate = 0.01) {
  # Initialize coefficients (weights) and intercept (bias)
  m <- nrow(X_train)
  n <- ncol(X_train)
  W <- rep(0, n)
  b <- 0

  # Gradient descent for Lasso regression
  for (i in seq_len(iterations)) {
    Y_pred <- X_train %*% W + b
    dW <- rep(0, n)
    
    # Update each weight
    for (j in seq_len(n)) {
      if (W[j] > 0) {
        dW[j] <- (-2 * sum(X_train[, j] * (y_train - Y_pred)) + alpha) / m
      } else {
        dW[j] <- (-2 * sum(X_train[, j] * (y_train - Y_pred)) - alpha) / m
      }
    }
    db <- -2 * sum(y_train - Y_pred) / m
    
    # Update weights and bias
    W <- W - learning_rate * dW
    b <- b - learning_rate * db
  }

  return(list(W = W, b = b))
}


In [22]:
# Load necessary library for plotting
library(ggplot2)

# Assume alphas, X_train, and y_train are defined and 'lasso_regression' function is available
# Set up k-fold cross-validation
set.seed(24)
kf <- createFolds(y_train, k = 5, list = TRUE)

avg_mse_values <- numeric(length(alphas))
names(avg_mse_values) <- as.character(alphas)
best_alpha <- NULL
min_avg_mse <- Inf

for (alpha in alphas) {
  mse_list <- numeric(length(kf))  # Initialize an empty list to keep track of the MSE for each fold
  
  for (i in seq_along(kf)) {
    train_index <- kf[[i]]
    val_index <- setdiff(seq_along(y_train), train_index)
    
    X_train_fold <- X_train[train_index, , drop = FALSE]
    X_val_fold <- X_train[val_index, , drop = FALSE]
    y_train_fold <- y_train[train_index]
    y_val_fold <- y_train[val_index]
    
    # Train Lasso regression model with the current alpha
    model_results <- lasso_regression(X_train_fold, y_train_fold, alpha)
    W <- model_results$W
    b <- model_results$b
    y_pred_val <- X_val_fold %*% W + b
    
    # Calculate MSE for this fold
    mse_fold <- mean((y_val_fold - y_pred_val)^2)
    mse_list[i] <- mse_fold
  }
  
  # Calculate average MSE across all folds
  avg_mse <- mean(mse_list)
  avg_mse_values[as.character(alpha)] <- avg_mse
  cat(sprintf("Alpha=%.1f, Average MSE: %.5f\n", alpha, avg_mse))
  
  # Update the best alpha and minimum average MSE
  if (avg_mse < min_avg_mse) {
    min_avg_mse <- avg_mse
    best_alpha <- alpha
  }
}

cat(sprintf("Best Alpha: %.1f, Minimum Average MSE: %.5f\n", best_alpha, min_avg_mse))

# Plotting the cross-validated MSE for each alpha value
df_plot <- data.frame(Alpha = as.numeric(names(avg_mse_values)), Avg_MSE = avg_mse_values)
ggplot(df_plot, aes(x = Alpha, y = Avg_MSE)) +
  geom_point() +
  geom_line() +
  labs(title = 'Cross-Validated MSE for Different Alpha Values', x = 'Alpha', y = 'Average MSE') +
  theme_minimal() +
  scale_x_continuous(breaks = as.numeric(names(avg_mse_values)))


ERROR: Error in library(ggplot2): there is no package called 'ggplot2'


In [ ]:

# Train the Lasso Regression Model
model_results <- lasso_regression(X_train, y_train, best_alpha)
W <- model_results$W
b <- model_results$b

# Make predictions on the test set
y_pred <- X_test %*% W + b

# Calculate the Pearson correlation coefficient
lasso_corr <- cor(y_test, y_pred)

# Calculate Mean Absolute Error (MAE)
lasso_mae <- mean(abs(y_test - y_pred))

# Print the results
cat(sprintf("Correlación: %.4f\n", lasso_corr))
cat(sprintf("MAE: %.4f\n", lasso_mae))
